In [1]:
import os
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

In [2]:
DATA_PATH = "../data/processed/financial_ratios_final_clean.csv"
OUT_DIR = "../outputs/tables"
os.makedirs(OUT_DIR, exist_ok=True)

df = pd.read_csv(DATA_PATH, encoding="utf-8-sig")

In [3]:
SIZE_CANDIDATES = [
    "ln_Total_Assets",
    "ln_Revenue"
]

In [4]:
def select_size_indicator_by_industry(df_industry):
    """
    Chọn biến quy mô đại diện cho 1 ngành
    dựa trên PCA loading của PC1
    """

    X = df_industry[SIZE_CANDIDATES].dropna()

    if X.shape[0] < 10:
        return None

    X_scaled = StandardScaler().fit_transform(X)

    pca = PCA(n_components=1)
    pca.fit(X_scaled)

    loadings = pd.Series(
        np.abs(pca.components_[0]),
        index=SIZE_CANDIDATES
    )

    return loadings.idxmax()

In [5]:
size_indicator_records = []

for industry, g in df.groupby("Ngành ICB - cấp 1"):
    selected_var = select_size_indicator_by_industry(g)

    if selected_var is None:
        continue

    size_indicator_records.append({
        "Ngành ICB - cấp 1": industry,
        "Selected_Size_Indicator": selected_var
    })

size_indicator_df = pd.DataFrame(size_indicator_records)
size_indicator_df

,Ngành ICB - cấp 1,Selected_Size_Indicator
0,Công nghiệp,ln_Revenue
1,Công nghệ Thông tin,ln_Total_Assets
2,Dược phẩm và Y tế,ln_Total_Assets
3,Dầu khí,ln_Revenue
4,Dịch vụ Tiêu dùng,ln_Total_Assets
5,Hàng Tiêu dùng,ln_Total_Assets
6,Nguyên vật liệu,ln_Total_Assets
7,Tiện ích Cộng đồng,ln_Total_Assets


In [6]:
size_indicator_path = os.path.join(
    OUT_DIR,
    "size_representative_indicator_by_industry.csv"
)
size_indicator_df.to_csv(
    size_indicator_path,
    index=False,
    encoding="utf-8-sig"
)

print("Saved:", size_indicator_path)

Saved: ../outputs/tables/size_representative_indicator_by_industry.csv


In [7]:
df_size = df.merge(
    size_indicator_df,
    on="Ngành ICB - cấp 1",
    how="left"
)

In [8]:
def classify_size_quantile(df_block):
    size_var = df_block["Selected_Size_Indicator"].iloc[0]

    if size_var not in df_block.columns:
        return None

    df_block["Size_Label"] = pd.qcut(
        df_block[size_var],
        q=3,
        labels=["Small", "Medium", "Large"]
    )
    return df_block

In [9]:
df_size_labeled = (
    df_size
    .dropna(subset=["Selected_Size_Indicator"])
    .groupby(["Ngành ICB - cấp 1", "Năm"], group_keys=False)
    .apply(classify_size_quantile)
)

/tmp/ipykernel_29429/1811871005.py:5: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(classify_size_quantile)


In [10]:
FINAL_COLS = [
    "Mã", "Tên công ty",
    "Ngành ICB - cấp 1",
    "Ngành ICB - cấp 2",
    "Ngành ICB - cấp 3",
    "Ngành ICB - cấp 4",
    "Năm",
    "Selected_Size_Indicator",
    "Size_Label"
]

df_size_final = df_size_labeled[FINAL_COLS].copy()
df_size_final.head()

,Mã,Tên công ty,Ngành ICB - cấp 1,Ngành ICB - cấp 2,Ngành ICB - cấp 3,Ngành ICB - cấp 4,Năm,Selected_Size_Indicator,Size_Label
0,AAA,An Phát Bioplastics,Nguyên vật liệu,Hóa chất,Hóa chất,"Nhựa, cao su & sợi",2020,ln_Total_Assets,Large
1,AAA,An Phát Bioplastics,Nguyên vật liệu,Hóa chất,Hóa chất,"Nhựa, cao su & sợi",2021,ln_Total_Assets,Large
2,AAA,An Phát Bioplastics,Nguyên vật liệu,Hóa chất,Hóa chất,"Nhựa, cao su & sợi",2022,ln_Total_Assets,Large
3,AAA,An Phát Bioplastics,Nguyên vật liệu,Hóa chất,Hóa chất,"Nhựa, cao su & sợi",2023,ln_Total_Assets,Large
4,AAA,An Phát Bioplastics,Nguyên vật liệu,Hóa chất,Hóa chất,"Nhựa, cao su & sợi",2024,ln_Total_Assets,Large


In [11]:
FINAL_PATH = os.path.join(
    OUT_DIR,
    "firm_size_classification_by_industry_year.csv"
)

df_size_final.to_csv(
    FINAL_PATH,
    index=False,
    encoding="utf-8-sig"
)

print("✅ Saved size classification to:", FINAL_PATH)

✅ Saved size classification to: ../outputs/tables/firm_size_classification_by_industry_year.csv
